In [3]:
from pymongo import MongoClient
import datetime
import pprint

### MongoDB 데이터 베이스 생성

In [24]:
conn =  MongoClient('mongodb://localhost:27017/')

In [25]:
mydb = conn.test

### 데이터베이스 존재 확인  
  
MongoDB에서 데이터베이스는 Content를 얻을 때까지 생성되지 않음
따라서, Collection과 Document를 먼저 생성해야지만, 데이터베이스의 존재를 확인할 수 있음

In [20]:
print(conn.list_database_names())

['admin', 'config', 'local']


### MongoDB Collection 생성
MongoDB 콜렉션(Collection)을 생성하기 위해, 데이터베이스 오브젝트를 사용하여 만들고자 하는 콜렉션 이름을 지정
MongoDB는 지정된 콜렉션 이름이 존재하지 않았다면, 바로 콜렉션을 생성함

*** MongoDB 콜렉션은 SQL Database인 RDBMS의 Table의 위치에 있음**

In [28]:
col = mydb.members

### Collection 존재 확인
MongoDB에서 콜렉션은 Content를 얻을 때까지 생성되지 않음  
따라서, Document를 먼저 생성해야지만, 콜렉션의 존재를 확인할 수 있음

In [29]:
print(mydb.list_collection_names())

[]


### 정보 삽입

In [30]:
people = {
    "이름" : "홍길동",
    "나이" : 30,
    "별명" : "의적"
}

In [31]:
col.insert(people)

<ipython-input-31-542bec4d8656>:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  col.insert(people)


ObjectId('61110d4faa946ed10589be5a')

### 검색(find)

In [32]:
result = col.find()

In [33]:
for r in result:
    print(r)

{'_id': ObjectId('61110d4faa946ed10589be5a'), '이름': '홍길동', '나이': 30, '별명': '의적'}


In [38]:
# 특정조건검색 / 이름이 홍길동인 것을 모두 가져옴
result = col.find({'이름': '홍길동'})
for r in result:
    print(r)

{'_id': ObjectId('61110d4faa946ed10589be5a'), '이름': '홍길동', '나이': 30, '별명': '의적'}


In [40]:
# 특정조건검색 / 이름이 홍길동이면서 나이가 30인 결과를 모두 가져옴 (and)
result = col.find({"이름":"홍길동","나이":30})
for r in result:
    print(r)

{'_id': ObjectId('61110d4faa946ed10589be5a'), '이름': '홍길동', '나이': 30, '별명': '의적'}


In [41]:
#특정조건검색 / 이름이 홍길동이거나 나이가 30이거나 결과를 모두 가져옴 (or)
result = col.find({"$or" : [ {"이름":"홍길동"} , {"나이":30} ] } )
for r in result:
    print(r)

{'_id': ObjectId('61110d4faa946ed10589be5a'), '이름': '홍길동', '나이': 30, '별명': '의적'}


In [42]:
#결과의 맨 위의 값 하나만 가져옴
result = col.find_one({"이름":"홍길동"})
print(result)

{'_id': ObjectId('61110d4faa946ed10589be5a'), '이름': '홍길동', '나이': 30, '별명': '의적'}


In [43]:
# 나이가 40보다 큰 사람을 찾음
result = col.find({"나이": {"$gt" : 40} })   #$gte = 크거나 같거나 (40도 포함됨)
for r in result :
    print(r)

In [44]:
#나이가 40보다 크고 50보다 작은 경우
result = col.find({"나이": {"$gt" : 40, "$lt": 50} })   #$lte = 작거나 같거나 (50도 포함됨)
for r in result :
    print(r)

In [47]:
#나이가 40보다 크고 50보다 작은 데이터의 나이만 보고 싶을때
result = col.find({"나이": {"$gt" : 40, "$lte": 50}},{"나이":True})
for r in result:
    print(r)

In [48]:
#나이가 40보다 크고 50보다 작거나 같은 데이터의 나이만 보고 싶을때 /단 id는 미출력
result = col.find({"나이": {"$gt" : 40, "$lte": 50}},{"_id":False, "나이":True})
for r in result:
    print(r)

In [49]:
#limit(4) 결과를 4개만 보겠다.
result = col.find().limit(4)
for r in result:
    print(r)

{'_id': ObjectId('61110d4faa946ed10589be5a'), '이름': '홍길동', '나이': 30, '별명': '의적'}


In [50]:
#skip(1) 첫 1개의 데이터는 보지 않는다.
result = col.find().skip(1).limit(4)
for r in result:
    print(r)

### 업데이트(Update)

In [57]:
#아래와 같은 쿼리는 이름이 바뀌나 나이, 별명 등의 정보는 사라지기때문에 주의해야함.
col.update({"이름":"홍길동"}, {"이름":"유길동"})


# 홍길동을 찾아 주길동으로 바꿔라. 맨 위에 있는 데이터만 수정됨.
col.update({"이름":"홍길동"}, {"$set":{"이름" :"주길동"}})


<ipython-input-57-2a9235223960>:2: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  col.update({"이름":"홍길동"}, {"이름":"유길동"})
<ipython-input-57-2a9235223960>:6: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  col.update({"이름":"홍길동"}, {"$set":{"이름" :"주길동"}})


{'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}

### 업서트(upsert)

In [58]:
col.update({"이름":"정길동"},{"$set":{"별명":"의적"}},upsert=True,multi=True)

<ipython-input-58-253a3b91fd70>:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  col.update({"이름":"정길동"},{"$set":{"별명":"의적"}},upsert=True,multi=True)


{'n': 1,
 'nModified': 0,
 'upserted': ObjectId('6111105cccefb37646b20167'),
 'ok': 1.0,
 'updatedExisting': False}

In [60]:
result = col.find()
for r in result:
    print(r)
    

{'_id': ObjectId('61110d4faa946ed10589be5a'), '이름': '유길동'}
{'_id': ObjectId('6111105cccefb37646b20167'), '이름': '정길동', '별명': '의적'}


### 삭제-1 (remove)

In [61]:
# 이름이 정길동인 데이터를 모두 삭제한다.
col.remove({"이름":"정길동"})

#데이터가 전체 삭제됨. 주의해야함
# col.remove({})

<ipython-input-61-65fff4febac1>:2: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  col.remove({"이름":"정길동"})


{'n': 1, 'ok': 1.0}

In [62]:
result = col.find()
for r in result:
    print(r)
    

{'_id': ObjectId('61110d4faa946ed10589be5a'), '이름': '유길동'}


### 삭제-2 (delete)

In [65]:
# 데이터삭제_2 - delete_one 하나만 삭제
col.delete_one({"이름":"주길동"})

#delete_many 전체삭제
col.delete_many({"이름":"유길동"})

In [66]:
result = col.find()
for r in result:
    print(r)
    